In [103]:
import pandas as pd
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn

In [104]:
df=pd.read_csv('100_Unique_QA_Dataset.csv')
df.head()

question      answer
0                   What is the capital of France?       Paris
1                  What is the capital of Germany?      Berlin
2               Who wrote 'To Kill a Mockingbird'?  Harper-Lee
3  What is the largest planet in our solar system?     Jupiter
4   What is the boiling point of water in Celsius?         100

In [105]:
def tokenize(text: str):
    text=text.lower()
    text=text.replace('?','')
    text=text.replace("'","")
    return text.split()

In [106]:
dictionary = {'<UNK>':0}

In [107]:
def preprocess(text):
    tokenized_question=tokenize(text['question'])
    tokenized_answer=tokenize(text['answer'])
    merged=tokenized_question+tokenized_answer
    for token in merged:
        if token not in dictionary:
            dictionary[token]=len(dictionary)

In [108]:
df.apply(preprocess,axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [109]:
def text_to_indices(text,vocab):
    indexed_text=[]
    for token in tokenize(text):
        if token in dictionary:
            indexed_text.append(dictionary[token])
        else:
            indexed_text.append('<UNK>')
    return indexed_text

In [110]:
text_to_indices("What is my name ?",dictionary)

[1, 2, '<UNK>', '<UNK>']

In [111]:
class QAdataset(Dataset):
    def __init__(self,df,dictionary):
        self.df=df
        self.dictionary=dictionary

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        numerical_question=text_to_indices(self.df.iloc[index]['question'],self.dictionary)
        numerical_answer=text_to_indices(self.df.iloc[index]['answer'],self.dictionary)
        return torch.tensor(numerical_question),torch.tensor(numerical_answer)

In [112]:
dataset=QAdataset(df,dictionary)

In [113]:
dataloader=DataLoader(dataset,batch_size=1,shuffle=True)

In [114]:
for question,answer in dataloader:
    print(question,answer)

tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([[52]])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([[68]])
tensor([[78, 79, 80, 81, 82, 83, 84]]) tensor([[85]])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([[36]])
tensor([[ 1,  2,  3, 37, 38, 39, 40]]) tensor([[41]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([[149]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([[106]])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([[6]])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([[154]])
tensor([[ 42, 174,   2,  62,  39, 175, 176,  12, 177, 178]]) tensor([[179]])
tensor([[ 42, 137,   2, 138,  39, 175, 269]]) tensor([[99]])
tensor([[  1,   2,   3,   4,   5, 279]]) tensor([[280]])
tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([[36]])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([[205]])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([[136]])
tensor([[42, 43, 44,

In [115]:
class MySimpleNN(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.embeddding=nn.Embedding(vocab_size,50)
        self.rnn=nn.RNN(50,64,batch_first=True)
        self.fc=nn.Linear(64,vocab_size)

    def forward(self,question):
        embedded_question=self.embeddding(question)
        hidden,final=self.rnn(embedded_question)
        output=self.fc(final.squeeze(0))

        return output

In [116]:
learning_rate=0.001
epochs=20
model=MySimpleNN(len(dictionary))
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [129]:
for epoch in range(epochs):
    total_loss=0
    for question,answer in dataloader:

        optimizer.zero_grad()
        
        output=model(question)
        
        loss=criterion(output,answer[0])
        
        loss.backward()
        
        optimizer.step()

        total_loss=total_loss+loss.item()
    print(f"Epoch: {epoch+1}, Loss:{total_loss:4f}")
        


Epoch: 1, Loss:0.029371
Epoch: 2, Loss:0.028023
Epoch: 3, Loss:0.026790
Epoch: 4, Loss:0.025616
Epoch: 5, Loss:0.024492
Epoch: 6, Loss:0.023420
Epoch: 7, Loss:0.022383
Epoch: 8, Loss:0.021391
Epoch: 9, Loss:0.020452
Epoch: 10, Loss:0.019545
Epoch: 11, Loss:0.018717
Epoch: 12, Loss:0.017875
Epoch: 13, Loss:0.017073
Epoch: 14, Loss:0.016324
Epoch: 15, Loss:0.015609
Epoch: 16, Loss:0.014926
Epoch: 17, Loss:0.014289
Epoch: 18, Loss:0.013634
Epoch: 19, Loss:0.013046
Epoch: 20, Loss:0.012478


In [120]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, dictionary)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(dictionary.keys())[index])

In [121]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [123]:
list(dictionary.keys())[7]

'paris'